<a href="https://colab.research.google.com/github/preetmodi/Credit-Risk-Analytics/blob/main/XGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo echo -ne '\n' | sudo add-apt-repository ppa:alessandro-strada/ppa >/dev/null 2>&1 # note: >/dev/null 2>&1 is used to supress printing
!sudo apt update >/dev/null 2>&1
!sudo apt install google-drive-ocamlfuse >/dev/null 2>&1
!google-drive-ocamlfuse
!sudo apt-get install w3m >/dev/null 2>&1 # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop >/dev/null 2>&1 # to set default browser 
%cd /content
!mkdir gdrive
%cd gdrive
!mkdir "My Drive"
!google-drive-ocamlfuse "/content/gdrive/My Drive"

In [ ]:
!pip install git+https://github.com/CBravoR/scorecardpy

%cd /content/gdrive/My Drive/Data/

In [ ]:
# Package loading
import pandas as pd
import numpy as np
import scorecardpy as sc
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
%matplotlib inline
seed = 251256517

In [ ]:
# Import the files as Pandas datasets
# train_WoE = pd.read_csv('train_woe.csv')
# test_WoE = pd.read_csv('test_woe.csv')

train_noWoE = pd.read_csv('train.csv')
test_noWoE = pd.read_csv('test.csv')

# Give breaks for WoE
with open('breaks_adj.json', 'r') as f:
    breaks_adj = json.load(f)
# breaks_adj = {'Address': [1.0,2.0,8.0,17.0],
#               'Age': [30.0,45.0,50.0],
#               'Creddebt': [1.0, 6.0],
#               'Employ': [4.0,14.0,22.0],
#               'Income': [30.0,40.0,80.0,140.0],
#               'Leverage': [8.0,16.0,22.0],
#               'MonthlyLoad': [0.1,0.2,0.30000000000000004,0.7000000000000001],
#               'OthDebt': [1.0,2.0,3.0],
#               'OthDebtRatio': [0.01,0.05,0.07,0.09,0.14]
#               }

# Apply breaks.
bins_adj = sc.woebin(train_noWoE, y="Default",
                     breaks_list=breaks_adj)
  

In [ ]:
from xgboost import XGBClassifier 
#Define the classifier.
XGB_lendingclub = XGBClassifier(max_depth=2,                 # Depth of each tree
                            learning_rate=0.1,            # How much to shrink error in each subsequent training. Trade-off with no. estimators.
                            n_estimators=50,             # How many trees to use, the more the better, but decrease learning rate if many used.
                            verbosity=1,                  # If to show more errors or not.
                            objective='binary:logistic',  # Type of target variable.
                            booster='gbtree',             # What to boost. Trees in this case.
                            n_jobs=2,                    # Parallel jobs to run. Set your processor number.
                            gamma=0.001,                  # Minimum loss reduction required to make a further partition on a leaf node of the tree. (Controls growth!)
                            subsample=0.632,              # Subsample ratio. Can set lower
                            colsample_bytree=1,           # Subsample ratio of columns when constructing each tree.
                            colsample_bylevel=1,          # Subsample ratio of columns when constructing each level. 0.33 is similar to random forest.
                            colsample_bynode=1,           # Subsample ratio of columns when constructing each split.
                            reg_alpha=1,                  # Regularizer for first fit. alpha = 1, lambda = 0 is LASSO.
                            reg_lambda=0,                 # Regularizer for first fit.
                            scale_pos_weight=1,           # Balancing of positive and negative weights. G / B
                            base_score=0.5,               # Global bias. Set to average of the target rate.
                            random_state=seed,        # Seed
                            tree_method='hist',           # How to train the trees?
                            #gpu_id=0                     # With which GPU? 
                            )

## Change these

In [ ]:
param_grid = dict({'n_estimators': [50, 100, 150],
                   'max_depth': [2, 3, 4],
                 'learning_rate' : [0.01, 0.05, 0.1, 0.15]
                  })

In [ ]:
# Always a good idea to tune on a reduce sample of the train set, as we will call many functions.
val_train = train_noWoE.sample(frac = 0.5,               # The fraction to extract
                                       random_state = seed,    # The seed.
                                       )

## Run Grid Search


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define grid search object.
GridXGB = GridSearchCV(XGB_lendingclub,        # Original XGB. 
                       param_grid,          # Parameter grid
                       cv = 3,              # Number of cross-validation folds.  
                       scoring = 'roc_auc', # How to rank outputs.
                       n_jobs = 2,          # Parallel jobs. -1 is "all you have"
                       refit = False,       # If refit at the end with the best. We'll do it manually.
                       verbose = 1          # If to show what it is doing.
                      )

# Follow the same approach for all categorical Variables

In [ ]:
categorical_features = ["Education"]
categorical_transformer = OneHotEncoder(handle_unknown="ignore")
preprocessor = ColumnTransformer(
    transformers=[("cat", categorical_transformer, categorical_features)],
    remainder='passthrough'
)

# Now we define a Pipeline to process everything
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", GridXGB)]
)

# Train the XGB.
clf.fit(val_train.drop(columns='Default'), # X 
        val_train['Default']    # y
        )

In [ ]:
# Show best params
print(f'The best AUC is {GridXGB.best_score_:.3f}')
GridXGB.best_params_

# Train model on entire training dataset

In [ ]:
# Create XGB with best parameters.
XGB_lendingclub = XGBClassifier(max_depth=GridXGB.best_params_.get('max_depth'), # Depth of each tree
                            learning_rate=GridXGB.best_params_.get('learning_rate'), # How much to shrink error in each subsequent training. Trade-off with no. estimators.
                            n_estimators=GridXGB.best_params_.get('n_estimators'), # How many trees to use, the more the better, but decrease learning rate if many used.
                            verbosity=1,                  # If to show more errors or not.
                            objective='binary:logistic',  # Type of target variable.
                            booster='gbtree',             # What to boost. Trees in this case.
                            n_jobs=2,                     # Parallel jobs to run. Set your processor number.
                            gamma=0.001,                  # Minimum loss reduction required to make a further partition on a leaf node of the tree. (Controls growth!)
                            subsample=0.632,              # Subsample ratio. Can set lower
                            colsample_bytree=1,           # Subsample ratio of columns when constructing each tree.
                            colsample_bylevel=1,          # Subsample ratio of columns when constructing each level. 0.33 is similar to random forest.
                            colsample_bynode=1,           # Subsample ratio of columns when constructing each split.
                            reg_alpha=1,                  # Regularizer for first fit. alpha = 1, lambda = 0 is LASSO.
                            reg_lambda=0,                 # Regularizer for first fit.
                            scale_pos_weight=1,           # Balancing of positive and negative weights.
                            base_score=0.5,               # Global bias. Set to average of the target rate.
                            random_state=20201107,        # Seed
                            tree_method='gpu_hist',       # How to train the trees?
                            gpu_id=0                      # With which GPU?
                            )

In [ ]:
# Train over all training data.
# Create dummy variables for education
categorical_features = ["Education"]
categorical_transformer = OneHotEncoder(handle_unknown="ignore")
preprocessor = ColumnTransformer(
    transformers=[("cat", categorical_transformer, categorical_features)],
    remainder='passthrough'
)

# Now we define a Pipeline to process everything
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", XGB_lendingclub)]
)

# Train the XGB.
clf.fit(train_noWoE.drop(columns='Default'), # X 
                  train_noWoE['Default']    # y
                  )

# Variable Importance

In [ ]:
# Plot variable importance
importances = XGB_lendingclub.feature_importances_
indices = np.argsort(importances)[::-1] 

f, ax = plt.subplots(figsize=(3, 8))
plt.title("Variable Importance - XGBoosting")
sns.set_color_codes("pastel")
sns.barplot(y=[clf[:-1].get_feature_names_out()[i] for i in indices], x=importances[indices], 
            label="Total", color="b")
ax.set(ylabel="Variable",
       xlabel="Variable Importance (Entropy)")
sns.despine(left=True, bottom=True)

### Confusion Matrix

In [ ]:
# Calculate probability
XGBClassTest = clf.predict(test_noWoE.drop(columns="Default"))
xg_probs_test = clf.predict_proba(test_noWoE.drop(columns="Default"))
xg_probs_test = xg_probs_test[:, 1]

# Calculate confusion matrix
confusion_matrix_xgb = confusion_matrix(y_true = test_noWoE['Default'], 
                    y_pred = XGBClassTest)

# Turn matrix to percentages
confusion_matrix_xgb = confusion_matrix_xgb.astype('float') / confusion_matrix_xgb.sum(axis=1)[:, np.newaxis]

# Turn to dataframe
df_cm = pd.DataFrame(
        confusion_matrix_xgb, index=['good', 'bad'], columns=['good', 'bad'], 
)

# Parameters of the image
figsize = (10,7)
fontsize=14

# Create image
fig = plt.figure(figsize=figsize)
heatmap = sns.heatmap(df_cm, annot=True, fmt='.2f')

# Make it nicer
heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, 
                             ha='right', fontsize=fontsize)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45,
                             ha='right', fontsize=fontsize)

# Add labels
plt.ylabel('True label')
plt.xlabel('Predicted label')

# Plot!
plt.show()

## ROC AUC

In [ ]:
# Calculate the ROC curve points
fpr, tpr, thresholds = roc_curve(test_noWoE['Default'], 
                                 xg_probs_test)

# Save the AUC in a variable to display it. Round it first
auc = np.round(roc_auc_score(y_true = test_noWoE['Default'], 
                             y_score = xg_probs_test),
               decimals = 3)

# Create and show the plot
plt.plot(fpr,tpr,label="AUC - XGBoosting = " + str(auc))
plt.legend(loc=4)
plt.show()

## Bootstapping to calculate error rates

In [ ]:
# Apply bootstrapping
auc_boot = BootstrapPredErr(pred_prob=xg_probs_test,  
                            y_true=test_noWoE['Default'])
upper=np.quantile(auc-auc_boot, 0.975, axis=0)
lower=np.quantile(auc-auc_boot, 0.025, axis=0)

# Compute confidence interval
boot_ci = [auc - upper, 
           auc - lower]
print(f"AUC Confidence Interval: [{boot_ci[0]:.3f}, {boot_ci[1]:.3f}]")

#Shap Values

In [ ]:
!pip install shap

In [ ]:
import shap
shap.initjs() # Import Java engine.

In [ ]:
# Create transformed data from previous preprocessor
data = pd.DataFrame(preprocessor.fit_transform(test_noWoE.drop(columns="Default")))
data.columns = preprocessor.get_feature_names_out()


# Trains the game-theoretic model. Really complex so requires sampling.
explainer = shap.TreeExplainer(XGB_lendingclub,                 # The model    
                              data = shap.sample(data, 100)  # Create a sample of 100 cases
                              )

# Applies model ot the full dataset.
shap_values = explainer.shap_values(data, check_additivity=False)

In [ ]:
shap.summary_plot(shap_values,                       # The Shapley values.
                  data, # The training sample
                  show=False)                        # Whether to print the model or not

# Let's save this as a PDF for later use.
plt.savefig('ShapSummaryPlot.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
shap.dependence_plot("remainder__Employ",                          # The variable to study
                     shap_values,                       # The Shapley values.
                     data, # The training sample
                     show=False)                        # Whether to print the model or not

plt.savefig('ShapEmploy.pdf', dpi=300, bbox_inches='tight')
plt.show()

# Compare Models

In [ ]:
# Predict probabilities of scorecard.
logreg_probs_test = logreg.predict_proba(test_WoE.iloc[:, 1:])

In [ ]:
# Set models and probabilities. This structure is called a dictionary.
models = [
{
    'label': 'Logistic Regression',
    'probs': logreg_probs_test[:,1]
},
{
    'label': 'Gradient Boosting',
    'probs': xg_probs_test
},


# Loop that creates the plot. I will pass each ROC curve one by one.
for m in models:
  auc = roc_auc_score(y_true = test_noWoE['Default'], 
                             y_score = m['probs'])
  fpr, tpr, thresholds = roc_curve(test_WoE['Default'], 
                                           m['probs'])
  plt.plot(fpr, tpr, label=f'{m["label"]} ROC (area = {auc:.3f})')
                 

    
# Settings
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
    
# Plot!    
plt.show()